In [1]:
from deep_translator import GoogleTranslator
from include.utils import *
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

import faiss
import numpy as np
import os
import pandas as pd
import torch

C:\Users\luiz.santos\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


**Configurações iniciais**

Definimos os caminhos para os arquivos de dados e índice FAISS e carregamos o DataFrame do arquivo Parquet.

In [2]:
# Configurações
diretorio_arquivos = '../arquivos'
parquet_file_path = os.path.join(diretorio_arquivos, 'trn.parquet')
diretorio_finetunning_modelo_final = os.path.join(diretorio_arquivos, 'finetunning','modelo_final')
faiss_index_path = os.path.join(diretorio_arquivos,'embeddings','amazon_products_index.faiss')

# Carregar DataFrame do Parquet
dados = load_dataframe(parquet_file_path)

**Configuração e Carregamento**

Configuramos o tradutor, o modelo SentenceTransformer e carregamos o modelo e tokenizer finetuned, bem como o índice FAISS.

In [3]:
# Configuração do tradutor
translator = GoogleTranslator(source='en', target='pt')

# Carregamento do modelo e tokenizer finetuned
print(f"Carregando modelo finetuned de {diretorio_finetunning_modelo_final}...")
tokenizer = DistilBertTokenizerFast.from_pretrained(diretorio_finetunning_modelo_final)
model = DistilBertForSequenceClassification.from_pretrained(diretorio_finetunning_modelo_final)

# Carregamento do índice FAISS
print(f"Carregando índice FAISS de {faiss_index_path}...")
faiss_index = faiss.read_index(faiss_index_path)
print(f"Índice FAISS carregado com {faiss_index.ntotal} vetores.")

Carregando modelo finetuned de ../arquivos\finetunning\modelo_final...
Carregando índice FAISS de ../arquivos\embeddings\amazon_products_index.faiss...
Índice FAISS carregado com 100 vetores.


**Processamento de Perguntas e Respostas**

Configuramos a função para dividir o contexto em blocos menores, classificar a relevância dos blocos baseados na pergunta, traduzir o texto, encontrar o bloco mais relevante e identificar os livros mais relevantes.

In [4]:
# Função para dividir o contexto em blocos menores
def split_context_into_blocks(context, block_size=100):
    words = context.split()
    return [' '.join(words[i:i + block_size]) for i in range(0, len(words), block_size)]


# Função para classificar a relevância dos blocos de um contexto baseado na pergunta
def classify_block_relevance(question, context, block_size=100):
    relevance_scores = []
    blocks = split_context_into_blocks(context, block_size)

    for block in blocks:
        inputs = tokenizer.encode_plus(question, block, return_tensors='pt', max_length=512, truncation=True)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Supondo que temos uma tarefa de classificação binária, pegamos a pontuação da classe positiva
        logits = outputs.logits
        score = torch.softmax(logits, dim=1).max(dim=1).values.item()  # Pegamos a probabilidade da classe mais provável
        relevance_scores.append((block, score))

    # Ordena os blocos com base na pontuação de relevância (do mais alto para o mais baixo)
    relevance_scores.sort(key=lambda x: x[1], reverse=True)
    return relevance_scores
    

# Função para traduzir o texto (por exemplo, usando um tradutor fictício)
def traduzir_texto(texto):
    try:
        return translator.translate(texto)
    except Exception as e:
        print(f"Erro na tradução: {e}")
        return texto
        

# Função para encontrar o bloco mais relevante e responder à pergunta
def find_most_relevant_block(question, context):
    relevance_scores = classify_block_relevance(question, context)
    
    if not relevance_scores:
        return "Nenhum bloco relevante encontrado."

    # O bloco mais relevante é o primeiro na lista após a ordenação
    best_block, best_score = relevance_scores[0]
    best_block_pt = traduzir_texto(best_block)  # Traduzir o bloco para o português
    return f"Bloco mais relevante (pontuação: {round(best_score, 2)}):\n{best_block_pt}"

def find_top_relevant_books(question, dataframe, top_n=3):
    all_relevance_scores = []

    # Use tqdm para adicionar uma barra de progresso
    for _, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0], desc="Processando livros"):
        titulo = row['title']
        contexto = row['content']
        relevance_scores = classify_block_relevance(question, contexto)
        if relevance_scores:
            best_block, best_score = relevance_scores[0]  # Melhor bloco
            all_relevance_scores.append((titulo, best_block, best_score))

    # Ordena todos os livros com base na pontuação de relevância (do mais alto para o mais baixo)
    all_relevance_scores.sort(key=lambda x: x[2], reverse=True)
    top_books = all_relevance_scores[:top_n]

    return top_books

**Teste e Exibição de Resultados**

Usamos uma pergunta específica para testar o código com uma pequena amostra de dados, encontrando o bloco mais relevante e traduzindo a resposta.

In [5]:
# Perguntas específicas
pergunta = "Quais são as principais características deste produto?"

# Exemplo de uso com dados do DataFrame
for _, row in dados.head(10).iterrows():  # Limitar a quantidade para teste
    titulo = row['title']
    contexto = row['content']
    
    print(f"\n--- Título: {titulo} ---")
    
    resposta = find_most_relevant_block(pergunta, contexto)
    print(f"\Contexto (us): {contexto}\n")
    print(f"Resposta (pt): {resposta}\n")


--- Título: Girls Ballet Tutu Neon Pink ---


<>:12: SyntaxWarning: invalid escape sequence '\C'
<>:12: SyntaxWarning: invalid escape sequence '\C'
C:\Users\luiz.santos\AppData\Local\Temp\ipykernel_20804\142051507.py:12: SyntaxWarning: invalid escape sequence '\C'
  print(f"\Contexto (us): {contexto}\n")


\Contexto (us): High quality 3 layer ballet tutu. 12 inches in length

Resposta (pt): Bloco mais relevante (pontuação: 0.5):
Tutu de balé de 3 camadas de alta qualidade. 12 polegadas de comprimento


--- Título: Mog's Kittens ---
\Contexto (us): Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.

Resposta (pt): Bloco mais relevante (pontuação: 0.5):
As aventuras mais vendidas de Judith Kerr daquele gato cativante (e exasperante) Mog têm entretido crianças por mais de 30 anos. Agora, até mesmo bebês e crianças pequenas podem se divertir conhecendo esse adorável felino. Esses pequenos livros cartonados resistentes, com suas imagens

Configuramos a pergunta para retornar os três melhores livros, processamos o DataFrame para encontrar os mais relevantes e exibimos os resultados com a descrição traduzida.

In [6]:
pergunta = "Poderia me retornar os três melhores livros ?"

# Exemplo de uso com dados do DataFrame
top_books = find_top_relevant_books(pergunta, dados)

# Exibir os melhores livros
for i, (titulo, bloco, score) in enumerate(top_books, 1):
    bloco_pt = traduzir_texto(bloco)
    print(f"\n--- Livro {i}: {titulo} ---")
    print(f"Relevância: {round(score, 2)}")
    print(f"Descrição traduzida: {bloco_pt}\n")

Processando livros:   1%|▊                                                                              | 16591/1498718 [47:38<70:56:32,  5.80it/s]


KeyboardInterrupt: 